# <span style = "color:purple"> <span style = "font-family: Times New Roman"> Hello & Bonjour! </span>

### <span style = "color:purple"><span style = "font-family: Times New Roman"> Welcome to French Learning 101 where we make LLMs are teacher to help become Fluent French Spokesperson. </span>


##### <span style = "color:purple"> <span style = "font-family: Times New Roman"> This tool helps you with the french translation for a given sentence and also helps in pronounciations along with suggestion of what should be your reply to the given sentence</span>


***

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import os

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [2]:
system_message = " You are a french teacher "

In [10]:
# to add audio for the chat

from pydub import AudioSegment
from pydub.playback import play
from io import BytesIO
import pygame

def talker(message):
    ''' Read the reponse and gives audio feedback pronouncing the response'''
    response = openai.audio.speech.create(
      model="tts-1",
      voice="alloy",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    pygame.init()
    pygame.mixer.init(frequency=int(44100*0.75))
    # Load the audio from the BytesIO object
    sound = pygame.mixer.Sound(audio_stream)
    # Play the sound
    sound.play()
    # Keep the program running until the sound finishes playing
    while pygame.mixer.get_busy():
        pygame.time.delay(100)
    # Quit Pygame
    pygame.quit()

In [4]:
def translate_to_french(message):
    ''' This function gives french translation of your sentence'''
    
    system_message = "You are a translator who reads the user message (which is in English) and tells the same sentence in French. "
    system_message += "For example, if the message is 'hello', say it like 'Bonjour'."
    
    messages = [{"role": "system", "content": system_message}] + [{"role": "user", "content": message}]
    
    try:
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {str(e)}"

    

In [5]:
def suggest_next_line(message):
    ''' This function suggests the reply to your sentence in french'''
    system_message = "You are a translator who reads the user message (which is in English) and suggests the best next response in French. Write the translation in english on the next line"
    system_message += "For example, if the message is 'How are you?', your response should be 'Je vais bien, merci. Translation: I'm doing well, thanks'"
    system_message += "For example, 'Please may i come in?' your response should be 'Oui, entrez! \n Translation: Yes, come in!'"
    messages = [{"role": "system", "content": system_message}] + [{"role": "user", "content": message}]
    
    try:
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {str(e)}"

In [6]:
def process_text(user_input):
    translation = translate_to_french(user_input)
    next_line = suggest_next_line(user_input)
    return translation, next_line

In [7]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
""" # ensuring dark theme!!

In [8]:
custom_css = """
    @import url('https://fonts.googleapis.com/css2?family=Great+Vibes&display=swap');

    body {
        background-color: #121212 !important;
        color: #ffffff !important;
        font-family: Times New Roman, sans-serif;
    }
    .gradio-container {
        max-width: 90%;
        margin: auto;
        padding: 10px;
        border-radius: 10px;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        background-color: #121212 !important;
        color: #ffffff !important;
        font-size: 30px;
    }
    .gradio-container h1 {
        text-align: center;
        font-size: 60px;
        font-weight: 700;
        font-style: italic;
        font-family: 'Great Vibes', cursive;  /* Elegant script font */
        text-transform: none;
        background: linear-gradient(to right, #ff0000, #ffffff, #000091); /* Red-White-Blue */
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        letter-spacing: 3px;        
        text-shadow: 2px 2px 5px rgba(255, 105, 180, 0.4); /* Subtle pink glow */
    }
    .gradio-container .gr-box {
        border: none !important;
        box-shadow: none !important;
    }
    .gradio-container .gr-box .label {
        display: none !important;
    }
    .gradio-container img {
        width: 100vw;
        height: auto;
        border-radius: 10px;
        background-color: transparent !important;
    }
    .gr-markdown {
    background-color: #121212 !important;
    color: #ffffff !important;
    }
    #custom_submit_button {
        background-color: #e01b1b !important; 
        color: white !important;
        font-weight: bold;
        padding: 10px 20px;
        border-radius: 8px;
        transition: 0.3s ease-in-out;
    }
    #custom_submit_button:hover {
        background-color: #ff3333 !important;
    }
    #custom_Pronounce_button {
        background-color: #ffffff !important; 
        color: #000000 !important;
        font-weight: bold;
        padding: 10px 20px;
        border-radius: 8px;
        transition: 0.3s ease-in-out;
    }
    #custom_Pronounce_button:hover {
        background-color: #d4d2d2 !important;
    }
    #custom_next_line_translation_button {
        background-color: #000091 !important;
        color: white !important;
        font-weight: bold;
        padding: 10px 20px;
        border-radius: 8px;
        transition: 0.3s ease-in-out;
    }
    #custom_next_line_translation_button:hover {
        background-color: #0000d1 !important;
    }
"""

In [9]:
with gr.Blocks(css=custom_css, js=force_dark_mode) as ui:    
    gr.Image("./header_img.png")
    gr.Markdown("# French Learning 101: Apprenons le français.")
    
    with gr.Row():
        user_input = gr.Textbox(label="Enter English Sentence",  lines=4)
        translation_output = gr.Textbox(label="French Translation", interactive=False,  lines=4)
        next_line_output = gr.Textbox(label="Next Line Suggestion", interactive=False,  lines=4)
    with gr.Row():
        submit_button = gr.Button("Translate & Suggest", elem_id="custom_submit_button")
        Pronounce_translation = gr.Button("Pronounciation", elem_id="custom_Pronounce_button")
        next_line_translation = gr.Button("Pronounciation", elem_id="custom_next_line_translation_button")
        
    submit_button.click(process_text, inputs=[user_input], outputs=[translation_output, next_line_output])
    Pronounce_translation.click(talker, inputs=[translation_output])
    next_line_translation.click(talker, inputs=[next_line_output])
# Launch the Gradio app
ui.launch()


* Running on local URL:  http://127.0.0.1:7913

To create a public link, set `share=True` in `launch()`.
